In [15]:
import json
import struct

In [2]:
with open("models/qwen/Qwen2-0.5B-Instruct/tokenizer.json", "r", encoding="utf-8") as f:
    tokenizer_config = json.load(f)

In [3]:
vocab = ["" for _ in range(151936)]

In [4]:
for k, v in tokenizer_config["model"]["vocab"].items():
    vocab[v] = k

In [5]:
for item in tokenizer_config["added_tokens"]:
    vocab[item["id"]] = item["content"]

In [8]:
max_len = max([i+1 for i, s in enumerate(vocab) if s])
print(max_len)

151646


In [11]:
scores = []
tokens = []
for s in vocab[:max_len]:
    scores.append(len(s) ** 2 + 0.0)
    t = s.replace("Ġ", " ")
    t = t.replace("Ċ", "\n")
    tokens.append(t.encode('utf-8'))
print(len(scores))
print(len(tokens))

151646
151646


In [17]:
print(151936 - 151646)

290


In [12]:
max_token_length = max(len(t) for t in tokens)
print(max_token_length)

128


In [13]:
tokenizer_bin = "qwen2_tokenizer.bin"

In [16]:
with open(tokenizer_bin, 'wb') as f:
    f.write(struct.pack("II", len(tokens), max_token_length))
    for bytes, score in zip(tokens, scores):
        f.write(struct.pack("fI", score, len(bytes)))
        f.write(bytes)

In [7]:
print(vocab[-100:])

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [6]:
print(k, v)

â½Ĺ 151642


In [10]:
with open("prompt_tokens.txt", "r") as fin:
    prompt_tokens = fin.read()
    id_list = [int(i) for i in prompt_tokens.split(" ")]
    print(id_list)

[29, 151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 35127, 752, 264, 2805, 16800, 311, 3460, 4128, 1614, 13, 151645, 198, 151644, 77091, 198]


In [11]:
str_list = []
for i in id_list[1:]:
    token = vocab[i]
    if not token:
        print(f"Empty for id: {i}.")
    str_list.append(token)

In [14]:
output = "".join(str_list).replace("Ġ", " ")
output = output.replace("Ċ", "\n")

In [16]:
print(ord("Ġ"))
print(ord(" "))
print(ord("Ċ"))
print(ord("\n"))

288
32
266
10


In [15]:
print(output)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant



In [17]:
max_id = 0
for i, v in enumerate(vocab):
    if v:
        max_id = i
print(max_id)

151645


In [18]:
with open("tokenizer_vocab.txt", "w", encoding="utf-8") as fout:
    for v in vocab[:151646]:
        fout.write(v + "\n")

In [ ]:
with open("tokenizer.bin", "wb") as f:
    for v in vocab:
        f.write(v.encode("utf-8"))
        f.write(b'\0')